In [12]:
import requests
import holidays
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.linear_model import Ridge

def compute_decomposition_factors(train_df, test_df, CFG):
    """
    A robust decomposition function that:
      1) merges train & test
      2) extracts date/time features
      3) computes GDP, store, product, weekday factors
      4) day-of-year factor via fourier
      5) extended sin/cos factor
      6) country factor
      7) non-periodic day-of-year factor
      8) final ratio = product of factors
      9) residual (train only)

    Missing data in any step => skip/ignore that factor or fill with 1.
    """
    # -------------------------
    # 1) Combine train + test
    # -------------------------
    train_df = train_df.copy()
    test_df  = test_df.copy()
    train_df['test'] = 0
    test_df['test']  = 1
    df = pd.concat([train_df, test_df], ignore_index=True)

    # -------------------------
    # 2) Basic date features
    # -------------------------
    df['date']      = pd.to_datetime(df['date'])
    df['year']      = df['date'].dt.year
    df['weekday']   = df['date'].dt.weekday
    df['dayofyear'] = df['date'].dt.dayofyear
    df['daynum']    = (df['date'] - df['date'].min()).dt.days
    df['weeknum']   = df['daynum'] // 7
    df['month']     = df['date'].dt.month

    # “daysinyear”: needed for sin/cos
    # If any year has 0 rows, fill with 1 to avoid division by zero
    group_y = df.groupby('year')['id'].count()
    group_y = group_y.replace({0:1})  # in case some year has 0 rows
    daysinyear = (group_y / (len(CFG.countries) * len(CFG.stores) * len(CFG.products))) \
                 .rename('daysinyear').astype(int).to_frame()
    df = df.merge(daysinyear, on='year', how='left')

    # Fill any missing daysinyear with median or 1
    df['daysinyear'] = df['daysinyear'].fillna(df['daysinyear'].median()).replace({0:1})

    # Initialize columns for each factor to 1 by default
    df['gdp_factor'] = 1.0
    df['store_factor'] = 1.0
    df['product_factor'] = 1.0
    df['weekday_factor'] = 1.0
    df['dayofyear_factor'] = 1.0
    df['sincos_factor'] = 1.0
    df['country_factor'] = 1.0
    df['npdoy_factor'] = 1.0

    # ----------------------------------------------------------------
    # 3) GDP Factor (country-year)
    # ----------------------------------------------------------------
    def get_gdp_per_capita(country, year):
        url = f"https://api.worldbank.org/v2/country/{CFG.alpha3[country]}/indicator/NY.GDP.PCAP.CD?date={year}&format=json"
        try:
            resp = requests.get(url).json()
            return float(resp[1][0]['value'])
        except:
            return np.nan

    gdp_table = {}
    for c in CFG.countries:
        for y in CFG.years:
            gdp_val = get_gdp_per_capita(c, y)
            gdp_table[(c, y)] = gdp_val

    for row_i in df.index:
        c = df.at[row_i, 'country']
        y = df.at[row_i, 'year']
        val = gdp_table.get((c,y), np.nan)
        if pd.notna(val):
            df.at[row_i, 'gdp_factor'] = val

    # Fill remaining NaNs in gdp_factor with median
    gdp_median = df['gdp_factor'].median(skipna=True)
    df['gdp_factor'] = df['gdp_factor'].fillna(gdp_median)

    # ----------------------------------------------------------------
    # 4) Store Factor
    # ----------------------------------------------------------------
    # Exclude countries with known issues (Canada, Kenya). Then group.
    mask_valid = ~df['country'].isin(['Canada','Kenya']) & (df['test'] == 0)
    store_mean = df[mask_valid].groupby('store')['num_sold'].mean()
    # fill with global store mean if missing
    store_mean = store_mean.fillna(store_mean.median())

    for s, val in store_mean.items():
        df.loc[df['store'] == s, 'store_factor'] = val
    # Normalize store_factor: e.g. divide by median so typical store=1
    med_store = df['store_factor'].median(skipna=True)
    df['store_factor'] = df['store_factor'].fillna(med_store).apply(lambda x: x/med_store)

    # ----------------------------------------------------------------
    # 5) Product Factor (Ridge on sin/cos)
    # ----------------------------------------------------------------
    df['partofyear']  = (df['dayofyear'] - 1) / df['daysinyear']
    df['partof2year'] = df['partofyear'] + (df['year'] % 2)

    df['sin t']   = np.sin(2*np.pi * df['partofyear'])
    df['cos t']   = np.cos(2*np.pi * df['partofyear'])
    df['sin t/2'] = np.sin(np.pi * df['partof2year'])
    df['cos t/2'] = np.cos(np.pi * df['partof2year'])

    sincos_cols = ['sin t','cos t','sin t/2','cos t/2']

    # We'll only compute product_factor for train rows & valid countries
    # Then fill test w/predicted or fallback=1 if not possible
    df_prod = df[(df['test'] == 0) & (~df['country'].isin(['Canada','Kenya']))].copy()
    # total sold by date => for ratio
    tot_by_date = df_prod.groupby('date')['num_sold'].sum().rename('daily_sum')
    df_prod = df_prod.merge(tot_by_date, on='date', how='left')
    df_prod['num_sold_ratio'] = df_prod['num_sold'] / df_prod['daily_sum']

    for prod in CFG.products:
        sub = df_prod[df_prod['product'] == prod].copy()
        if sub.empty:
            continue
        # X => sin/cos
        X = sub[sincos_cols].values
        y = sub['num_sold_ratio'].values

        # Drop NaN rows
        valid = (~np.isnan(X).any(axis=1)) & (~np.isnan(y))
        X = X[valid]
        y = y[valid]
        if len(X) < 2:  # not enough data to fit
            continue

        # Fit
        reg = Ridge()
        try:
            reg.fit(X,y)
        except ValueError:
            continue

        # Predict for all rows with this product
        mask_prod = (df['product'] == prod)
        X_pred = df.loc[mask_prod, sincos_cols].fillna(0).values
        df.loc[mask_prod, 'product_factor'] = reg.predict(X_pred)

    # product_factor => typical scale ~1
    # So we normalize by median
    med_pf = df['product_factor'].median(skipna=True)
    if pd.notna(med_pf) and med_pf!=0:
        df['product_factor'] = df['product_factor'].fillna(med_pf).div(med_pf)
    else:
        df['product_factor'] = 1.0  # fallback

    # ----------------------------------------------------------------
    # 6) Weekday Factor
    # ----------------------------------------------------------------
    # We'll compute a median ratio by weekday
    # ignoring holiday. We'll define holiday below.
    df['holiday'] = 0
    for c in CFG.countries:
        c_hols = holidays.CountryHoliday(CFG.countries_2l[c], years=CFG.years)
        hdays  = [pd.Timestamp(str(d)) for d in c_hols.keys()]
        mask_c = (df['country'] == c) & (df['date'].isin(hdays))
        df.loc[mask_c, 'holiday'] = 1

    # If you want to exclude holiday entirely:
    no_hol_mask = (df['holiday'] == 0) & (df['test'] == 0)
    weekday_ratio = df[no_hol_mask].groupby('weekday')['num_sold'].sum()
    total_sum = weekday_ratio.sum()
    if total_sum > 0:
        weekday_ratio = weekday_ratio / total_sum
        # apply to df
        for w, val in weekday_ratio.items():
            df.loc[df['weekday'] == w, 'weekday_factor'] = val
    # Ensure no NaN
    df['weekday_factor'] = df['weekday_factor'].fillna(df['weekday_factor'].median())

    # ----------------------------------------------------------------
    # 7) Day-of-year Factor (Fourier)
    # ----------------------------------------------------------------
    # partial_ratio => gdp*store*product*weekday
    df['partial_ratio'] = (
        df['gdp_factor'] *
        df['store_factor'] *
        df['product_factor'] *
        df['weekday_factor']
    )
    # avoid div-zero
    df['partial_ratio'] = df['partial_ratio'].replace({0:np.nan})
    df['temp_total'] = df['num_sold'] / df['partial_ratio']

    # exclude holidays
    df['holiday_response'] = 0
    for c in CFG.countries:
        c_hols = holidays.CountryHoliday(CFG.countries_2l[c], years=CFG.years)
        for hol_day, _ in c_hols.items():
            rng = pd.date_range(hol_day, periods=CFG.holiday_response_len)
            mask_hr = (df['country']==c) & (df['date'].isin(rng))
            df.loc[mask_hr, 'holiday_response'] = 1

    # dayofyear => median of temp_total
    sub_doy = df[(df['holiday_response']==0) & (df['test']==0)]
    dayofyear_median = sub_doy.groupby('dayofyear')['temp_total'].median()
    dayofyear_median = dayofyear_median.dropna()
    if dayofyear_median.empty:
        # skip dayofyear_factor
        df['dayofyear_factor'] = 1.0
    else:
        xvals = dayofyear_median.index.values
        yvals = dayofyear_median.values
        # Basic 1-frequency Fourier
        def fourier_func(t):
            return np.vstack([np.sin(2*np.pi/365.0 * t),
                              np.cos(2*np.pi/365.0 * t)]).T
        # Filter out NaN
        valid = (~np.isnan(xvals)) & (~np.isnan(yvals))
        xvals = xvals[valid]
        yvals = yvals[valid]
        if len(xvals)>2:
            reg_doy = Ridge(alpha=0.01)
            try:
                reg_doy.fit(fourier_func(xvals), yvals)
                # predict for t=1..366
                t_seq = np.arange(1,367)
                yoy_pred = reg_doy.predict(fourier_func(t_seq))
                # map to df
                yoy_map = dict(zip(t_seq, yoy_pred))
                df['dayofyear_factor'] = df['dayofyear'].map(yoy_map).fillna(1.0)
            except ValueError:
                df['dayofyear_factor'] = 1.0
        else:
            df['dayofyear_factor'] = 1.0

    # ----------------------------------------------------------------
    # 8) Extended sincos factor
    # ----------------------------------------------------------------
    df['sin 2t'] = np.sin(4*np.pi*df['partofyear'])
    df['cos 2t'] = np.cos(4*np.pi*df['partofyear'])
    df['sin 3t'] = np.sin(6*np.pi*df['partofyear'])
    df['cos 3t'] = np.cos(6*np.pi*df['partofyear'])
    df['sin 4t'] = np.sin(8*np.pi*df['partofyear'])
    df['cos 4t'] = np.cos(8*np.pi*df['partofyear'])
    sincoscol2 = ['sin 4t','cos 4t','sin 3t','cos 3t','sin 2t','cos 2t','sin t','cos t','sin t/2','cos t/2']

    # median of temp_total by date => ridges
    sub2 = df[(df['test']==0) & (df['holiday_response']==0)].copy()
    daily_median = sub2.groupby('date')['temp_total'].median().rename('median')
    # join sincos
    sc_merged = pd.merge(
        df[['date']+sincoscol2].drop_duplicates(),
        daily_median, on='date', how='left'
    )
    sc_merged = sc_merged.dropna(subset=['median'])
    if len(sc_merged)>2:
        X_sc = sc_merged[sincoscol2].values
        y_sc = sc_merged['median'].values
        valid2 = (~np.isnan(X_sc).any(axis=1)) & (~np.isnan(y_sc))
        X_sc = X_sc[valid2]
        y_sc = y_sc[valid2]
        if len(X_sc)>2:
            reg_sc = Ridge(alpha=0.01, fit_intercept=True)
            try:
                reg_sc.fit(X_sc, y_sc)
                # compute sincos_factor for entire df
                mat = df[sincoscol2].fillna(0).values
                sc_pred = reg_sc.intercept_ + np.sum(mat * reg_sc.coef_, axis=1)
                df['sincos_factor'] = sc_pred
            except ValueError:
                pass

    # normalize sincos_factor
    med_sf = df['sincos_factor'].median(skipna=True)
    if pd.notna(med_sf) and med_sf!=0:
        df['sincos_factor'] = df['sincos_factor'].fillna(med_sf) / med_sf
    else:
        df['sincos_factor'] = 1.0

    # ----------------------------------------------------------------
    # 9) Country factor
    # ----------------------------------------------------------------
    # partial ratio (no country factor yet)
    partial_ratio2 = (
        df['gdp_factor'] *
        df['store_factor'] *
        df['product_factor'] *
        df['weekday_factor'] *
        df['sincos_factor'] *
        df['dayofyear_factor']
    ).replace({0:np.nan})
    df['temp_total2'] = df['num_sold'] / partial_ratio2

    # e.g. pick product='Kaggle Mug'
    mask_kagg = (df['test']==0) & (df['product']=='Kaggle Mug')
    if mask_kagg.sum()>0:
        c_sums = df[mask_kagg].groupby('country')['temp_total2'].sum().rename('country_factor')
        c_sums = c_sums.dropna()
        if not c_sums.empty and c_sums.median()!=0:
            c_sums = c_sums / c_sums.median()
            for c_, val_ in c_sums.items():
                df.loc[df['country']==c_, 'country_factor'] = val_

    # fill missing country_factor => 1
    df['country_factor'] = df['country_factor'].fillna(1.0)

    # ----------------------------------------------------------------
    # 10) Non-periodic day-of-year factor
    # ----------------------------------------------------------------
    part3 = (
        df['gdp_factor'] *
        df['store_factor'] *
        df['product_factor'] *
        df['weekday_factor'] *
        df['sincos_factor'] *
        df['country_factor'] *
        df['dayofyear_factor']
    ).replace({0:np.nan})
    df['temp_total3'] = df['num_sold']/part3
    group_npdoy = df[df['test']==0].groupby('dayofyear')['temp_total3'].median()
    group_npdoy = group_npdoy.dropna()
    if not group_npdoy.empty:
        # map
        for d_oy, val_ in group_npdoy.items():
            df.loc[df['dayofyear']==d_oy, 'npdoy_factor'] = val_/group_npdoy.median()
    # fill missing => 1
    df['npdoy_factor'] = df['npdoy_factor'].fillna(1.0)

    # ----------------------------------------------------------------
    # 11) Final ratio
    # ----------------------------------------------------------------
    df['ratio'] = (
        df['gdp_factor'] *
        df['store_factor'] *
        df['product_factor'] *
        df['weekday_factor'] *
        df['dayofyear_factor'] *
        df['sincos_factor'] *
        df['country_factor'] *
        df['npdoy_factor']
    ).replace({0:np.nan})
    df['ratio'] = df['ratio'].fillna(1.0)

    # ----------------------------------------------------------------
    # 12) residual
    # ----------------------------------------------------------------
    train_mask = (df['test']==0)
    df.loc[train_mask, 'residual'] = df.loc[train_mask, 'num_sold'] / df.loc[train_mask, 'ratio']
    df.loc[~train_mask, 'residual'] = np.nan  # test => unknown

    # Drop intermediate columns if you like
    drop_cols = ['daysinyear','holiday','holiday_response','temp_total','temp_total2','temp_total3']
    for col in drop_cols:
        if col in df.columns:
            df.drop(col, axis=1, inplace=True, errors='ignore')

    return df


In [21]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import numpy as np
import pandas as pd

# Example config (matching your article's approach)
class CFG:
    countries = ["Finland", "Canada", "Italy", "Kenya", "Singapore", "Norway"]
    stores = ["KaggleMart", "KaggleRama"]  # example
    products = ["Kaggle Mug", "Kaggle Hat", "Kaggle Sticker"]  # example
    years_train = [2015, 2016, 2017]
    years_test = [2018]
    years = years_train + years_test
    alpha3 = {"Finland":"FIN","Canada":"CAN","Italy":"IT","Kenya":"KEN","Singapore":"SGP","Norway":"NOR"}
    countries_2l = {"Finland": "FI", "Canada": "CA", "Italy": "IT", "Kenya": "KE", "Singapore": "SG", "Norway": "NO"}
    # The columns used for product factor
    sincoscol = ["sin t","cos t","sin t/2","cos t/2"]
    holiday_response_len = 3  # days


###############################
# 1) Factor computation
###############################
from functools import partial
# Suppose we've put the function compute_decomposition_factors in a separate .py
# For simplicity, we define it inline or you can import it.
compute_factors = compute_decomposition_factors

###############################
# 2) FAN definition
###############################
# We'll use your minimal FAN for demonstration.
from layers.SelfAttention_Family import FullAttention, AttentionLayer
from layers.Transformer_EncDec import Decoder, DecoderLayer, Encoder, EncoderLayer
from layers.Embed import DataEmbedding
from layers.FANLayer import FANLayer

class FANModel(nn.Module):
    """Minimal demo: single-step seq, reusing your FAN logic."""
    def __init__(self, feature_dim=8, d_model=64):
        super().__init__()
        self.d_model = d_model

        class DummyConfigs:
            enc_in = feature_dim
            dec_in = feature_dim
            c_out = 1
            d_model = 64
            embed = 'timeF'
            freq = 'h'
            dropout = 0.1
            e_layers = 3
            d_layers = 1
            d_ff = 128
            n_heads = 4
            factor = 5
            activation = 'gelu'
            output_attention = False
            pred_len = 1
            exp_setting = 0

        configs = DummyConfigs()

        self.enc_embedding = DataEmbedding(
            configs.enc_in, configs.d_model, configs.embed, configs.freq, configs.dropout
        )
        self.dec_embedding = DataEmbedding(
            configs.dec_in, configs.d_model, configs.embed, configs.freq, configs.dropout
        )

        self.fan_layer = FANLayer(configs.d_model, configs.d_model)

        # Encoder
        self.encoder = Encoder(
            [
                EncoderLayer(
                    AttentionLayer(
                        FullAttention(
                            False,
                            configs.factor,
                            attention_dropout=configs.dropout,
                            output_attention=configs.output_attention
                        ),
                        configs.d_model,
                        configs.n_heads,
                    ),
                    configs.d_model,
                    configs.d_ff,
                    dropout=configs.dropout,
                    activation=configs.activation,
                    exp_setting=configs.exp_setting
                )
                for _ in range(configs.e_layers)
            ],
            norm_layer=nn.LayerNorm(configs.d_model),
        )

        # Decoder
        self.decoder = Decoder(
            [
                DecoderLayer(
                    AttentionLayer(
                        FullAttention(
                            True,
                            configs.factor,
                            attention_dropout=configs.dropout,
                            output_attention=False,
                        ),
                        configs.d_model,
                        configs.n_heads,
                    ),
                    AttentionLayer(
                        FullAttention(
                            False,
                            configs.factor,
                            attention_dropout=configs.dropout,
                            output_attention=False,
                        ),
                        configs.d_model,
                        configs.n_heads,
                    ),
                    configs.d_model,
                    configs.d_ff,
                    dropout=configs.dropout,
                    activation=configs.activation,
                    exp_setting=configs.exp_setting,
                )
                for _ in range(configs.d_layers)
            ],
            norm_layer=nn.LayerNorm(configs.d_model),
            projection=nn.Linear(configs.d_model, 1, bias=True),
        )

    def forward(self, x):
        # x shape [B, feature_dim]
        B, F = x.shape
        x_enc = x.unsqueeze(1)  # => [B,1,F]
        x_dec = x.unsqueeze(1)
        x_mark_enc = torch.zeros(B,1,4, device=x.device)
        x_mark_dec = torch.zeros(B,1,4, device=x.device)

        enc_out = self.enc_embedding(x_enc, x_mark_enc)
        enc_out = self.fan_layer(enc_out)
        enc_out, _ = self.encoder(enc_out)

        dec_out = self.dec_embedding(x_dec, x_mark_dec)
        dec_out = self.decoder(dec_out, enc_out)
        return dec_out.squeeze(1).squeeze(-1)  # => [B]

###############################
# 3) Torch Dataset
###############################
class ResidualDataset(Dataset):
    """
    We want to predict 'residual' from some chosen input features (like dayofyear, ratio, etc.)
    For train samples, y=residual is known. For test samples, y = NaN => we won't use them in training.
    """
    def __init__(self, df, feature_cols, target_col):
        # Filter only rows that have a valid residual (train only)
        self.df = df[~df[target_col].isna()].copy()
        self.X = self.df[feature_cols].values.astype(np.float32)
        self.y = self.df[target_col].values.astype(np.float32)

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        return (
            torch.tensor(self.X[idx], dtype=torch.float32),
            torch.tensor(self.y[idx], dtype=torch.float32),
        )

###############################
# 4) Main
###############################
def main():
    # 1) Load data
    # STEP 1: Load real Kaggle data
    TRAIN_CSV = "playground-series-s5e1/train.csv"
    TEST_CSV  = "playground-series-s5e1/test.csv"

    train_df = pd.read_csv(TRAIN_CSV)
    test_df  = pd.read_csv(TEST_CSV)

    # The 'train_df' should have columns: id, country, store, product, date, num_sold
    # The 'test_df'  should have columns: id, country, store, product, date

    # Convert `date` to datetime and mark test=0/1
    train_df['date'] = pd.to_datetime(train_df['date'])
    train_df['test'] = 0

    test_df['date']  = pd.to_datetime(test_df['date'])
    test_df['test']  = 1

    # STEP 2: Combine train+test
    df = pd.concat([train_df, test_df], ignore_index=True)

    # 2) Compute all factors + ratio + residual
    df_all = compute_factors(train_df, test_df, CFG)

    # 3) Split back into train_df, test_df
    train_df = df_all[df_all['test'] == 0].copy()
    test_df  = df_all[df_all['test'] == 1].copy()

    # ------------------------------------------
    # Choose features to feed the FAN model:
    # e.g. dayofyear, ratio, weekday, etc.
    # You can add more advanced features (sin/cos, holiday flags, etc.)
    # ------------------------------------------
    feature_cols = ['dayofyear','weekday','ratio']  # minimal example
    target_col   = 'residual'

    # 4) Build PyTorch Datasets
    train_dataset = ResidualDataset(train_df, feature_cols, target_col)
    print("Training samples:", len(train_dataset))

    # 5) DataLoader
    train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True)

    # 6) Initialize FAN
    torch.cuda.empty_cache()
    input_dim = len(feature_cols)
    model = FANModel(feature_dim=input_dim, d_model=32)
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model = model.to(device)
    optimizer = optim.Adam(model.parameters(), lr=1e-3)
    criterion = nn.MSELoss()

    # 7) Train
    epochs = 2
    model.train()
    for epoch in range(epochs):
        epoch_loss = 0
        for batch_x, batch_y in train_loader:
            batch_x = batch_x.to(device)
            batch_y = batch_y.to(device)

            optimizer.zero_grad()
            pred_y = model(batch_x)
            loss = criterion(pred_y, batch_y)
            loss.backward()
            optimizer.step()
            epoch_loss += loss.item()*len(batch_x)
        epoch_loss /= len(train_dataset)
        print(f"[Epoch {epoch+1}/{epochs}] MSE Loss: {epoch_loss:.6f}")

    # 8) Inference on test
    # Build the features for test
    test_X = torch.tensor(test_df[feature_cols].values.astype(np.float32), device=device)
    model.eval()
    with torch.no_grad():
        pred_residual = model(test_X).cpu().numpy()  # shape [test_size]

    # 9) Final prediction = ratio * predicted_residual
    test_ratio = test_df['ratio'].values
    test_preds = test_ratio * pred_residual

    # (Optionally) you can multiply by the 1.06 factor or do rounding, etc.
    final_preds = np.round(test_preds).astype(int)

    # 10) Build submission
    submission = pd.DataFrame({
        "id": test_df["id"].values,
        "num_sold": final_preds
    })
    submission.to_csv("playground-series-s5e1/fanforecast/submission.csv", index=False)
    print("Saved submission.csv!")
    print(submission.head(5))

if __name__ == "__main__":
    main()


Training samples: 221259
[Epoch 1/2] MSE Loss: 68.263383
[Epoch 2/2] MSE Loss: 68.216184
Saved submission.csv!
       id  num_sold
0  230130   -952856
1  230131   -952856
2  230132   -952856
3  230133   -952856
4  230134   -952856
